In [ ]:
import cv2
import os
from pathlib import Path
from matplotlib import pyplot as plt
import math
import numpy as np
from tqdm import tqdm

In [ ]:
path = Path("Dataset_TP3")
videos_paths = [p for p in path.iterdir() if p.is_file()]

videos: dict[Path, cv2.VideoCapture] = {}
for file in videos_paths:
    vid = cv2.VideoCapture(file)
    if not vid.isOpened():
        print("Error: Could not open video.")
        continue
    videos[file] = vid

In [ ]:
interactive = False
r = 50
edge_threshold = 30

edges = {}

print(f"Processing {len(videos)} videos")
for video_path, video in videos.items():
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))

    edge_maps = np.zeros((frame_count, h, w), dtype=np.uint8)
    histograms = np.zeros((frame_count, r), dtype=np.float32)

    center_y, center_x = h // 2, w // 2
    y, x = np.ogrid[:h, :w]
    distances = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)
    bin_size = np.max(distances) / r

    bin_indices = np.clip((distances / bin_size).astype(np.int32), 0, r - 1)

    for frame_idx in tqdm(range(frame_count)):
        ret, frame = video.read()
        if not ret:
            print(f"Error: Could not read frame {frame_idx}.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        sobel_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)

        magnitude = cv2.magnitude(sobel_x, sobel_y)
        edge_maps[frame_idx] = (magnitude > edge_threshold).astype(np.uint8)

        for i in range(r):
            mask = bin_indices == i
            if np.any(mask):
                histograms[frame_idx, i] = np.mean(magnitude[mask])

        if interactive:
            display = cv2.convertScaleAbs(magnitude)
            cv2.imshow("Edges", display)
            if cv2.waitKey(int(1000 / fps)) & 0xFF == ord("q"):
                break

    edges[video_path] = {"edge_maps": edge_maps, "histograms": histograms}
    video.set(cv2.CAP_PROP_POS_FRAMES, 0)

In [ ]:
def dilate_edges(edge_map, kernel_size=3):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    return cv2.dilate(edge_map, kernel)


for video_name, video_data in edges.items():
    edge_maps = video_data["edge_maps"]
    frame_count = edge_maps.shape[0]

    print(f"Processing {frame_count} frames for video {video_name.name}")

    rho_in_list = []
    rho_out_list = []

    for t in tqdm(range(frame_count - 1)):
        E_t = edge_maps[t]
        E_t1 = edge_maps[t + 1]

        D_t = dilate_edges(E_t)
        D_t1 = dilate_edges(E_t1)

        sum_E_t1 = np.sum(E_t1)
        if sum_E_t1 > 0:
            rho_in = 1 - np.sum(D_t & E_t1) / sum_E_t1
        else:
            rho_in = 0

        sum_E_t = np.sum(E_t)
        if sum_E_t > 0:
            rho_out = 1 - np.sum(E_t & D_t1) / sum_E_t
        else:
            rho_out = 0

        rho_in_list.append(rho_in)
        rho_out_list.append(rho_out)

    print(f"Video: {video_name.name}")
    print(
        f"Mean ρin: {np.mean(rho_in_list):.4f}, Mean ρout: {np.mean(rho_out_list):.4f}"
    )

    plt.figure(figsize=(12, 4))
    plt.plot(rho_in_list, label="ρin (entering edges)")
    plt.plot(rho_out_list, label="ρout (exiting edges)")
    plt.xlabel("Frame")
    plt.ylabel("ρ")
    plt.legend()
    plt.title(f"Edge Change Metrics - {video_name.name}")
    plt.show()

In [ ]:
def detect_transitions(
    rho_in_list, rho_out_list, cut_threshold=0.7, gradual_threshold=0.3, min_duration=5
):
    rho_in = np.array(rho_in_list)
    rho_out = np.array(rho_out_list)
    rho_max = np.maximum(rho_in, rho_out)

    cuts = []
    fades = []
    gradations = []

    for t in range(len(rho_max)):
        if rho_max[t] > cut_threshold:
            is_isolated = True
            if t > 0 and rho_max[t - 1] > gradual_threshold:
                is_isolated = False
            if t < len(rho_max) - 1 and rho_max[t + 1] > gradual_threshold:
                is_isolated = False
            if is_isolated:
                cuts.append(t)

    in_transition = False
    transition_start = 0

    for t in range(len(rho_max)):
        if not in_transition and rho_max[t] > gradual_threshold:
            in_transition = True
            transition_start = t
        elif in_transition and rho_max[t] <= gradual_threshold:
            in_transition = False
            duration = t - transition_start

            if duration >= min_duration:
                segment_in = rho_in[transition_start:t]
                segment_out = rho_out[transition_start:t]

                first_half = len(segment_in) // 2
                in_dominant_start = np.mean(segment_in[:first_half]) > np.mean(
                    segment_out[:first_half]
                )
                out_dominant_end = np.mean(segment_out[first_half:]) > np.mean(
                    segment_in[first_half:]
                )

                if in_dominant_start and out_dominant_end:
                    fades.append((transition_start, t))
                else:
                    gradations.append((transition_start, t))

    if in_transition and (len(rho_max) - transition_start) >= min_duration:
        t = len(rho_max)
        segment_in = rho_in[transition_start:t]
        segment_out = rho_out[transition_start:t]
        first_half = len(segment_in) // 2
        in_dominant_start = np.mean(segment_in[:first_half]) > np.mean(
            segment_out[:first_half]
        )
        out_dominant_end = np.mean(segment_out[first_half:]) > np.mean(
            segment_in[first_half:]
        )

        if in_dominant_start and out_dominant_end:
            fades.append((transition_start, t))
        else:
            gradations.append((transition_start, t))

    return cuts, fades, gradations


for video_name, video_data in edges.items():
    edge_maps = video_data["edge_maps"]
    frame_count = edge_maps.shape[0]

    rho_in_list = []
    rho_out_list = []

    for t in range(frame_count - 1):
        E_t = edge_maps[t]
        E_t1 = edge_maps[t + 1]
        D_t = dilate_edges(E_t)
        D_t1 = dilate_edges(E_t1)

        sum_E_t1 = np.sum(E_t1)
        rho_in = 1 - np.sum(D_t & E_t1) / sum_E_t1 if sum_E_t1 > 0 else 0

        sum_E_t = np.sum(E_t)
        rho_out = 1 - np.sum(E_t & D_t1) / sum_E_t if sum_E_t > 0 else 0

        rho_in_list.append(rho_in)
        rho_out_list.append(rho_out)

    cuts, fades, gradations = detect_transitions(rho_in_list, rho_out_list)

    print(f"\n=== {video_name.name} ===")
    print(f"Cuts detected: {len(cuts)} at frames {cuts}")
    print(f"Fades detected: {len(fades)} - {fades}")
    print(f"Gradations detected: {len(gradations)} - {gradations}")

    plt.figure(figsize=(14, 5))
    rho_max = np.maximum(rho_in_list, rho_out_list)

    plt.plot(rho_in_list, label="ρin (entering edges)", alpha=0.7)
    plt.plot(rho_out_list, label="ρout (exiting edges)", alpha=0.7)
    plt.plot(rho_max, label="ρmax", color="black", linewidth=1.5)

    for c in cuts:
        plt.axvline(
            x=c,
            color="red",
            linestyle="--",
            linewidth=2,
            label="Cut" if c == cuts[0] else "",
        )

    for i, (start, end) in enumerate(fades):
        plt.axvspan(start, end, alpha=0.3, color="blue", label="Fade" if i == 0 else "")

    for i, (start, end) in enumerate(gradations):
        plt.axvspan(
            start, end, alpha=0.3, color="green", label="Gradation" if i == 0 else ""
        )

    plt.xlabel("Frame")
    plt.ylabel("ρ")
    plt.legend()
    plt.title(f"Scene Transition Detection - {video_name.name}")
    plt.tight_layout()
    plt.show()

In [ ]:
DETECTION_THRESHOLD = 1
cuts_path = Path("./cuts")

detected_cuts = {}

for video_name, video_data in edges.items():
    edge_maps = video_data["edge_maps"]
    frame_count = edge_maps.shape[0]

    rho_in_list = []
    rho_out_list = []

    for t in range(frame_count - 1):
        E_t = edge_maps[t]
        E_t1 = edge_maps[t + 1]
        D_t = dilate_edges(E_t)
        D_t1 = dilate_edges(E_t1)

        sum_E_t1 = np.sum(E_t1)
        rho_in = 1 - np.sum(D_t & E_t1) / sum_E_t1 if sum_E_t1 > 0 else 0

        sum_E_t = np.sum(E_t)
        rho_out = 1 - np.sum(E_t & D_t1) / sum_E_t if sum_E_t > 0 else 0

        rho_in_list.append(rho_in)
        rho_out_list.append(rho_out)

    cuts, fades, gradations = detect_transitions(rho_in_list, rho_out_list)
    detected_cuts[video_name] = cuts

for video_name, gen_cuts in detected_cuts.items():
    name_file = video_name.name.split(".")[0]
    valid_cuts_path = cuts_path / f"{name_file}.txt"

    if not valid_cuts_path.exists():
        print(f"Ground truth file not found: {valid_cuts_path}")
        continue

    with open(valid_cuts_path, "r") as f:
        valid_cuts_str = f.read()

    valid_cuts = []
    for x in valid_cuts_str.split(","):
        stripped = x.strip()
        if stripped.isdigit():
            valid_cuts.append(int(stripped))

    correct_cuts = []
    incorrect_cuts = []

    for gc in gen_cuts:
        found = False
        for vc in valid_cuts:
            if abs(gc - vc) <= DETECTION_THRESHOLD:
                found = True
                break
        if found:
            correct_cuts.append(gc)
        else:
            incorrect_cuts.append(gc)

    TP = len(correct_cuts)
    FP = len(incorrect_cuts)

    missed_cuts = []
    for vc in valid_cuts:
        detected = any(abs(gc - vc) <= DETECTION_THRESHOLD for gc in gen_cuts)
        if not detected:
            missed_cuts.append(vc)
    FN = len(missed_cuts)

    precision = TP / (TP + FP) * 100 if (TP + FP) > 0 else 0.0

    recall = TP / (TP + FN) * 100 if (TP + FN) > 0 else 0.0

    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0.0

    total_detected = len(gen_cuts)

    print(f"\n{'=' * 50}")
    print(f"Vidéo: {video_name.name}")
    print(f"{'=' * 50}")
    print(f"Cuts détectés: {total_detected} | Ground truth: {len(valid_cuts)}")
    print(f"\n✓ Détections correctes (TP): {TP}")
    print(f"  Frames: {correct_cuts}")
    print(f"\n✗ Faux positifs (FP): {FP}")
    print(f"  Frames: {incorrect_cuts}")
    print(f"\n⊘ Cuts manqués (FN): {FN}")
    print(f"  Frames: {missed_cuts}")
    print(f"\n{'─' * 50}")
    print(f"Métriques:")
    print(f"   Précision: {precision:.1f}%")
    print(f"   Rappel:    {recall:.1f}%")
    print(f"   F1 Score:  {f1_score:.1f}%")